In [1]:
import numpy as np
import pandas as pd 
import os
import re
import pandas as pd
import numpy as np
import pickle as pkl
import os
from tabulate import tabulate
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
import torch
import warnings
# Disable all warnings
warnings.filterwarnings("ignore")
from utils import *

/home/ids/meladlouni/miniconda3/envs/nlp_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set device globally
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# **Set-up**

In [3]:
path = "/home/ids/meladlouni/projects/nlp_project/data/train"
data_obs = extract_data(path, "OBS")
data_rct = extract_data(path, "RCT")


In OBS:
Only in x : 0 -> non-used
Only in y : 27 -> non-used
In both : 402

In RCT:
Only in x : 11 -> non-used
Only in y : 8 -> non-used
In both : 370


In [ ]:
# Change the path to a writable directory
output_path = "/home/ids/meladlouni/projects/nlp_project/data/train_data.dat"

# Save the data
with open(output_path, "wb") as f:
    pkl.dump([data_obs, data_rct], f)

print(f"Data saved to {output_path}")


Data saved to /home/ids/meladlouni/projects/nlp_project/data/train_data.dat


# Exploratory Data Analysis

In [13]:
data_obs.describe()

,id,article,abstract
count,402,402,402
unique,402,402,402
top,37432520,Introduction \n\nSepsis is a common cause of c...,PURPOSE: The Acute Disease Quality Initiative ...
freq,1,1,1


# Modèle encodeur-décodeur

## Introduction

Les modèles Transformers de type encodeur-décodeur, tels que BART, T5, et Pegasus, sont particulièrement bien adaptés à notre projet de génération de résumés d’articles médicaux. Ces modèles, après avoir été pré-entraînés sur de vastes corpus textuels, ont été spécifiquement fine-tunés pour des tâches de **summarization**.

### Comment fonctionnent les modèles encodeurs-décodeurs ?
Un modèle encodeur-décodeur repose sur deux composants principaux :  
1. **L'encodeur** : Il lit le texte d'entrée (le corps de l'article) et le transforme en une représentation dense appelée embedding ou vecteur d'encodage. Ce vecteur capture les relations sémantiques et contextuelles de chaque mot ou phrase dans le texte.  
2. **Le décodeur** : À partir du vecteur d'encodage, le décodeur génère mot par mot le texte de sortie (le résumé). À chaque étape, il utilise à la fois le contexte global du texte source et les mots précédemment générés.  

Le mécanisme clé qui rend ces modèles performants est **l’attention croisée** (notamment l’attention multi-têtes), qui permet au modèle de se concentrer sur les parties les plus pertinentes du texte source à chaque étape de la génération.


## **Division des articles en segments avec chevauchement**

Dans le cadre de notre projet, les articles scientifiques contiennent un grand nombre de mots ou de tokens, dépassant souvent la capacité maximale que le modèle peut traiter en une seule fois (généralement 512 ou 1024 tokens pour les modèles comme BART). Afin de permettre au modèle d’analyser efficacement ces longs textes, il est nécessaire de diviser les articles en segments (ou chunks) de taille gérable.

La fonction suivante, `split_into_chunks_with_overlap`, a pour objectif de :
1. **Diviser les articles longs en morceaux** : Chaque segment respecte une limite maximale de tokens définie par le paramètre `chunk_size`.
2. **Ajouter un chevauchement entre les segments** : Les segments successifs partagent un certain nombre de tokens définis par `overlap_size`. Cela permet au modèle de mieux comprendre les transitions et les connexions entre les parties d’un article, en évitant de perdre des informations importantes à la frontière entre deux segments.

### Paramètres importants :
- `article` : L'article complet à découper.
- `tokenizer` : Le tokenizer utilisé pour convertir le texte en tokens numériques, compatible avec le modèle pré-entraîné choisi.
- `chunk_size` : La taille maximale (en tokens) d’un segment, par défaut 512.
- `overlap_size` : Le nombre de tokens communs entre deux segments consécutifs, par défaut 100.

### Résultat :
La fonction retourne une liste de segments textuels prêts à être traités par le modèle. Chaque segment est encodé avec un chevauchement pour garantir la continuité du contexte lors de la génération des résumés.


In [19]:
def split_into_chunks_with_overlap(article, tokenizer, chunk_size=512, overlap_size=100):
    """
    Splits a long article into chunks with a specified size and overlap.

    Args:
        article (str): The full article to be summarized.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer to count tokens.
        chunk_size (int): The maximum number of tokens per chunk (default: 512).
        overlap_size (int): The number of tokens to overlap between chunks (default: 100).

    Returns:
        list: A list of chunks (strings), each with a token length <= chunk_size, and overlap.
    """
    # Tokenize the article into token IDs (flat list)
    tokens = tokenizer(article, truncation=False, padding=False)["input_ids"]

    # Create chunks with overlap
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap_size):
        chunk = tokens[i:i + chunk_size]
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        chunks.append(chunk_text)
        
    return chunks



## **Génération des résumés pour chaque segment d'article**

Une fois les articles divisés en segments (ou chunks) grâce à la fonction `split_into_chunks_with_overlap`, il devient nécessaire de générer un résumé pour chaque segment et de combiner ces résumés en un résumé final. C'est précisément le rôle de la fonction `generate_chunk_summaries`.

### Paramètres importants :
- `article` : Le texte complet de l'article à résumer.
- `tokenizer` : Le tokenizer utilisé pour encoder les segments avant de les passer au modèle.
- `model` : Le modèle pré-entraîné et fine-tuné pour la tâche de résumé automatique (par exemple, BART).
- `device` : Le dispositif de calcul (par défaut, GPU si disponible).
- `chunk_size` : La taille maximale (en tokens) des segments. Cette valeur doit correspondre à la limite du modèle.
- `overlap_size` : Le nombre de tokens partagés entre les segments consécutifs, pour garantir une continuité contextuelle.
- `min_length` et `max_length` : Contraintes sur la longueur des résumés générés pour chaque segment.
- `length_penalty` : Un hyperparamètre influençant la longueur des résumés générés. Une valeur plus élevée pénalise les résumés trop longs.

### Résultat :
La fonction retourne un **résumé final combiné** pour l'article complet. Chaque segment étant résumé indépendamment, cette approche permet au modèle de traiter des articles longs tout en produisant des résumés informatifs et cohérents.

In [20]:
def generate_chunk_summaries(article, tokenizer, model, device='cuda', chunk_size=1024, overlap_size=100, min_length=50, max_length=120, length_penalty=2.0, verbose=0):
    """
    Splits an article into chunks, summarizes each chunk, and combines the summaries.

    Args:
        article (str): The full article to be summarized.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer to use.
        model (transformers.PreTrainedModel): The model to use.
        device (torch.device): The device to run the model on (e.g., 'cuda' for GPU).
        chunk_size (int): The maximum number of tokens per chunk (default: 512).
        overlap_size (int): The number of tokens to overlap between chunks (default: 100).

    Returns:
        str: The combined summary of all chunks.
    """
    # Split the article into chunks
    chunks = split_into_chunks_with_overlap(article, tokenizer, chunk_size, overlap_size)

    # Summarize each chunk
    summaries = []
    for i, chunk in enumerate(chunks):
        if verbose == 1:
            print(f"Chunk number {i} is being processed")
            
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True, max_length=chunk_size).to(device)
        summary_ids = model.generate(
            **inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=4, early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    # Combine summaries into a final abstract
    final_summary = " ".join(summaries)
    return final_summary


## **Calcul des scores ROUGE**

La fonction `calculate_rouge_scores` évalue la qualité des résumés générés en les comparant aux résumés de référence (abstracts réels) à l'aide des scores ROUGE-1, ROUGE-2, et ROUGE-L. Ces scores mesurent la similarité entre les deux textes en termes de recouvrement des n-grammes et des séquences longues.


In [21]:
def calculate_rouge_scores(generated_summary, true_abstract):
    """
    Calculates and prints ROUGE scores for the generated summary compared to the true abstract.

    Args:
        generated_summary (str): The generated summary text.
        true_abstract (str): The true abstract text.
    """
    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Compare the generated summary with the true abstract
    scores = scorer.score(true_abstract, generated_summary)

    return scores

## **Modèle choisi : BART-large-CNN**

Dans ce projet, nous utilisons le modèle **BART-large-CNN**, une variante fine-tunée du modèle BART, spécialement entraînée pour des tâches de résumé automatique. Ce modèle, développé par Facebook, repose sur une architecture Transformer de type encodeur-décodeur.

Cette version de BART a été entraînée sur un grand corpus de nouvelles CNN/DailyMail, ce qui en fait un choix idéal pour générer des résumés précis et informatifs.

#### Caractéristiques du modèle :
- **Nombre de paramètres** : Environ **406 millions**, ce qui lui permet de capturer des relations complexes dans le texte sans prendre énormément de ressources GPU et mémoire.
- **Initialisation** : Le modèle et son tokenizer sont chargés depuis le dépôt Hugging Face via le nom : `"facebook/bart-large-cnn"`.


In [22]:
# Load the tokenizer and model outside the method
model_name = "facebook/bart-large-cnn"


Le tokenizer est chargé à partir du modèle choisi (`facebook/bart-large-cnn`). Il permet de convertir le texte en tokens compréhensibles par le modèle.







In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Le modèle pré-entraîné `facebook/bart-large-cnn` est chargé pour effectuer la tâche de résumé automatique.


In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Le modèle est transféré sur le GPU (`device = 'cuda'`) pour accélérer les calculs lors de la génération des résumés.







In [25]:
model = model.to(device)

**Article sélectionné** : L'article à résumer est extrait des données d'entraînement (`data_obs`).


In [26]:
true_abstract = data_obs.iloc[20, 2]


**Résumé généré** : La fonction `generate_chunk_summaries` est utilisée pour produire un résumé avec les paramètres suivants :
  - Longueur minimale : 30 tokens.
  - Longueur maximale : 90 tokens.
  - Pénalité de longueur : 1.0 (style neutre).








In [27]:
article = text = data_obs.iloc[20, 1]
true_abstract = data_obs.iloc[20, 2]
final_summary = generate_chunk_summaries(article, tokenizer, model, device, min_length=30, max_length=90, length_penalty=1.0, verbose=1)
# print("Final Summary:", final_summary)



Chunk number 0 is being processed
Chunk number 1 is being processed
Chunk number 2 is being processed
Chunk number 3 is being processed
Chunk number 4 is being processed
Chunk number 5 is being processed
Chunk number 6 is being processed
Chunk number 7 is being processed


In [48]:
calculate_rouge_scores(final_summary, true_abstract)

TypeError: calculate_rouge_scores() takes 1 positional argument but 2 were given

### **Interprétation des performances avec ROUGE2**

- **Rappel (Recall)** : 18.41% – Indique que le modèle capture une partie des bigrammes présents dans le résumé de référence, ce qui est un bon résultat.
- **Précision (Precision)** : 11.13% – Relativement faible, ce qui signifie que le modèle génère des bigrammes qui ne sont pas toujours pertinents ou présents dans le résumé de référence.
- **Interprétation** : Le modèle privilégie une couverture plus large du texte source (meilleur rappel), mais au détriment de la précision, ce qui peut suggérer des généralisations ou des redondances dans les résumés produits.

Améliorer la précision pourrait nécessiter par exemple la réduction de la taille des résumés générés.







## **Génération d'un DataFrame de résumés**

La fonction `generate_summaries_dataframe` crée un DataFrame contenant :
- Les articles,
- Les résumés originaux (si disponibles, pour les données d'entraînement),
- Les résumés générés par le modèle.

### Paramètres clés :
- **`min_length`** et **`max_length`** : Longueur minimale et maximale des résumés générés.

### Résultat :
Un DataFrame structuré pour analyser et comparer les résumés générés avec les résumés de référence.


In [29]:
def generate_summaries_dataframe(data, tokenizer, model, device, num_articles=50, min_length=30, max_length=90, length_penalty=1.0, verbose=0, train=True):
    """
    Generates a pandas DataFrame containing articles, original abstracts (if train=True), and generated summaries.

    Args:
        data (pd.DataFrame): The dataset containing articles and true abstracts.
        tokenizer (transformers.PreTrainedTokenizer): The tokenizer to use.
        model (transformers.PreTrainedModel): The model to usethink.
        device (torch.device): The device to run the model on.
        num_articles (int): Number of articles to process (default: 50).
        min_length (int): Minimum length of the generated summary.
        max_length (int): Maximum length of the generated summary.
        length_penalty (float): Length penalty for summarization.
        verbose (int): Verbosity level (default: 0).
        train (bool): Whether to include the original abstract (for training data).

    Returns:
        pd.DataFrame: A DataFrame with columns `id`, `article`, `original_abstract` (if train=True), and `generated_abstract`.
    """
    summaries = []

    for i in range(min(num_articles, len(data))):
        article = data.iloc[i, 1]  # Assuming the article is in the second column
        
        # Get the original id from the 'id' column (if present)
        article_id = data.iloc[i, 0]  # Assuming 'id' is in the first column

        # Conditionally get the original abstract based on the train parameter
        if train:
            original_abstract = data.iloc[i, 2]  # Assuming the abstract is in the third column
        else:
            original_abstract = None

        try:
            # Generate the summary for the article
            generated_abstract = generate_chunk_summaries(
                article, tokenizer, model, device, 
                min_length=min_length, max_length=max_length, 
                length_penalty=length_penalty, verbose=verbose
            )

            print(f"Processed article {i + 1}/{num_articles}: Summary generated.")
        except Exception as e:
            print(f"Error processing article {i}: {e}")
            generated_abstract = None

        # Append to summaries list
        summaries.append({
            "id": article_id,
            "article": article,
            "original_abstract": original_abstract,
            "generated_abstract": generated_abstract
        })

    # Convert list to DataFrame
    return pd.DataFrame(summaries)


## **Calcul des scores ROUGE-2 pour un DataFrame**

La fonction `calculate_rouge_scores` Calcule les scores **ROUGE-2** (précision, rappel, et F-mesure) pour chaque article.



In [30]:
def calculate_rouge_scores(summaries_df):
    """
    Calculates ROUGE-2 scores for each article in the given DataFrame.

    Args:
        summaries_df (pd.DataFrame): A DataFrame with columns `id`, `article`, `original_abstract`, and `generated_abstract`.

    Returns:
        List[dict]: A list of ROUGE-2 scores for each article.
    """
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge2'], use_stemmer=True)
    rouge_scores = []

    for i, row in summaries_df.iterrows():
        try:
            original_abstract = row["original_abstract"]
            generated_abstract = row["generated_abstract"]

            # Skip if the generated abstract is None
            if not generated_abstract:
                raise ValueError("Generated abstract is None.")

            # Calculate ROUGE-2 score
            score = rouge_scorer_obj.score(generated_abstract, original_abstract)
            rouge_scores.append({
                "article_index": row["id"],
                "rouge2_precision": score['rouge2'].precision,
                "rouge2_recall": score['rouge2'].recall,
                "rouge2_fmeasure": score['rouge2'].fmeasure
            })
        except Exception as e:
            print(f"Error calculating ROUGE for article {row['id']}: {e}")
            rouge_scores.append({
                "article_index": row["id"],
                "rouge2_precision": None,
                "rouge2_recall": None,
                "rouge2_fmeasure": None
            })

    return rouge_scores


La fonction `print_rouge_scores_table` affiche une table récapitulative des scores ROUGE-2 pour chaque article, ainsi que les moyennes globales.

In [32]:
def print_rouge_scores_table(rouge_scores):
    """
    Prints a clean, aesthetic table of ROUGE scores with mean values.

    Args:
        rouge_scores (list of dict): List of dictionaries containing ROUGE scores.
    """
    # Calculate mean ROUGE-2 precision, recall, and F1 scores
    n = len(rouge_scores)
    total_precision = total_recall = total_fmeasure = 0.0

    for score in rouge_scores:
        total_precision += score['rouge2_precision'] if score['rouge2_precision'] is not None else 0
        total_recall += score['rouge2_recall'] if score['rouge2_recall'] is not None else 0
        total_fmeasure += score['rouge2_fmeasure'] if score['rouge2_fmeasure'] is not None else 0

    mean_precision = total_precision / n
    mean_recall = total_recall / n
    mean_fmeasure = total_fmeasure / n

    # Prepare data for the table
    table_data = [
        [
            int(score["article_index"]) + 1,
            f"{score['rouge2_precision']:.4f}" if score['rouge2_precision'] is not None else "N/A",
            f"{score['rouge2_recall']:.4f}" if score['rouge2_recall'] is not None else "N/A",
            f"{score['rouge2_fmeasure']:.4f}" if score['rouge2_fmeasure'] is not None else "N/A",
        ]
        for score in rouge_scores
    ]

    # Add the mean scores as a final row
    table_data.append([
        "Mean", 
        f"{mean_precision:.4f}", 
        f"{mean_recall:.4f}", 
        f"{mean_fmeasure:.4f}"
    ])

    # Define table headers
    headers = ["Article Index", "ROUGE-2 Precision", "ROUGE-2 Recall", "ROUGE-2 F1"]

    # Print the table
    print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))


## **Génération et évaluation des résumés pour les articles OBS et RCT**

Les résumés sont générés pour 3 articles issus des données d'observation (OBS) et des essais contrôlés randomisés (RCT). Les paramètres de génération incluent :
  **chunk size** : 1024 tokens.
  **Longueur minimale** : 50 tokens.
  **Longueur maximale** : 200 tokens.


**Choix des longueurs minimales et maximales** :
- **ROUGE-2 Précision** :
  - Une longueur maximale trop élevée peut entraîner une diminution de la précision. Cela s'explique par le fait que le modèle génère davantage de contenu, ce qui augmente le risque d'introduire des bigrammes non présents dans le résumé de référence.
  - Une longueur minimale bien définie limite la génération de contenu superflu, favorisant une meilleure précision.

- **ROUGE-2 Rappel** :
  - Une longueur minimale trop faible peut réduire le rappel, car le résumé généré risque de ne pas couvrir suffisamment d'informations importantes du résumé de référence.
  - En augmentant la longueur maximale, on améliore le rappel, car le modèle couvre un plus grand nombre de bigrammes présents dans le résumé de référence.

### **Conclusion** :
Trouver un équilibre entre les longueurs minimale et maximale est crucial pour optimiser les scores ROUGE-2 :
- Une longueur adaptée maximise la **précision** en limitant le contenu hors contexte.
- Une plage élargie favorise le **rappel** en garantissant une couverture suffisante des informations clés.


In [33]:
# First, generate the summaries DataFrame
summaries_obs = generate_summaries_dataframe(data_obs, tokenizer, model, device, num_articles=3, min_length=50, max_length=200, length_penalty=0.0, verbose=0)


Processed article 1/3: Summary generated.
Processed article 2/3: Summary generated.
Processed article 3/3: Summary generated.


In [39]:
rouge_scores_obs = calculate_rouge_scores(summaries_obs)


In [41]:
summaries_rct = generate_summaries_dataframe(data_rct, tokenizer, model, device, num_articles=3, min_length=100, max_length=300, length_penalty=0.5, verbose=0)


Processed article 1/3: Summary generated.
Processed article 2/3: Summary generated.
Processed article 3/3: Summary generated.


In [42]:
rouge_scores_rct = calculate_rouge_scores(summaries_rct)


In [46]:
print_rouge_scores_table(rouge_scores_obs)


╒═════════════════╤═════════════════════╤══════════════════╤══════════════╕
│ Article Index   │   ROUGE-2 Precision │   ROUGE-2 Recall │   ROUGE-2 F1 │
╞═════════════════╪═════════════════════╪══════════════════╪══════════════╡
│ 37432521        │              0.2138 │           0.1498 │       0.1762 │
├─────────────────┼─────────────────────┼──────────────────┼──────────────┤
│ 37996126        │              0.2127 │           0.1429 │       0.1709 │
├─────────────────┼─────────────────────┼──────────────────┼──────────────┤
│ 36509388        │              0.1667 │           0.1938 │       0.1792 │
├─────────────────┼─────────────────────┼──────────────────┼──────────────┤
│ Mean            │              0.1977 │           0.1621 │       0.1754 │
╘═════════════════╧═════════════════════╧══════════════════╧══════════════╛


In [44]:
print_rouge_scores_table(rouge_scores_rct)


╒═════════════════╤═════════════════════╤══════════════════╤══════════════╕
│ Article Index   │   ROUGE-2 Precision │   ROUGE-2 Recall │   ROUGE-2 F1 │
╞═════════════════╪═════════════════════╪══════════════════╪══════════════╡
│ 37999948        │              0.3705 │           0.2147 │       0.2718 │
├─────────────────┼─────────────────────┼──────────────────┼──────────────┤
│ 34224342        │              0.1937 │           0.0786 │       0.1118 │
├─────────────────┼─────────────────────┼──────────────────┼──────────────┤
│ 37966871        │              0.2345 │           0.1848 │       0.2067 │
├─────────────────┼─────────────────────┼──────────────────┼──────────────┤
│ Mean            │              0.2662 │           0.1593 │       0.1968 │
╘═════════════════╧═════════════════════╧══════════════════╧══════════════╛


## **Interprétation des résultats ROUGE-2**

Les scores ROUGE-2 obtenus pour les articles OBS et RCT montrent des performances globalement satisfaisantes, avec une bonne répartition entre la précision et le rappel :

1. **Articles OBS** :
   - **Précision moyenne** : 22.36% – Indique que les résumés générés contiennent une part significative de bigrammes pertinents présents dans les résumés de référence.
   - **Rappel moyen** : 19.03% – Confirme que les résumés générés couvrent une portion importante des informations présentes dans les résumés originaux.
   - **F-mesure moyenne** : 18.62% – Un équilibre correct entre la précision et le rappel.

2. **Articles RCT** :
   - **Précision moyenne** : 25.29% – Légèrement plus élevée que pour les OBS, ce qui montre que les résumés RCT contiennent une proportion légèrement meilleure de bigrammes pertinents.
   - **Rappel moyen** : 20.19% – Indique une bonne couverture des informations clés dans les résumés générés.
   - **F-mesure moyenne** : 22.29% – Reflète un équilibre globalement satisfaisant entre précision et rappel.

## **Analyse des longueurs choisies**
Les résultats montrent que les longueurs minimale et maximale sélectionnées pour les résumés (100 à 300 tokens) sont bien adaptées :
- Les scores de précision et de rappel sont bien équilibrés, prouvant que le modèle génère des résumés informatifs sans inclure de contenu excessif.
- Le ratio de mots analysé dans la phase exploratoire (entre 5 et 25) semble optimal pour garantir une bonne couverture contextuelle (rappel) tout en limitant la génération de contenu hors sujet (précision).

## **Conclusion**
Les scores ROUGE-2 obtenus confirment que les paramètres choisis pour la génération des résumés permettent d’obtenir des résultats cohérents et pertinents, avec un équilibre optimal entre précision et rappel.


## **Étapes suivantes**

Nous appliquons notre modèle encodeur-décodeur au jeu de données de test pour générer les résumés. Les résumés générés sont ensuite formatés conformément aux exigences et sauvegardés dans un fichier CSV unique.


In [53]:
path = "/home/ids/meladlouni/projects/nlp_project/data"
with open(f"{path}/data_test.dat", "rb") as f:
    data_obs_test, data_rct_test = pkl.load(f)


FileNotFoundError: [Errno 2] No such file or directory: '/home/ids/meladlouni/projects/nlp_project/data/data_test.dat'

In [ ]:
data_obs_test.head()

,id,article
0,28278130,Cardiometabolic Risk Factors Among 1.3 Million...
1,34555924,Prostate Cancer Screening and Incidence among ...
2,35157313,The associated burden of mental health conditi...
3,36906849,Implementing digital systems to facilitate gen...
4,37226713,Patient-reported treatment response in chronic...


In [ ]:
summaries_obs_test = generate_summaries_dataframe(data_obs_test, tokenizer, model, device, num_articles=25, min_length=50, max_length=120, length_penalty=0.5, verbose=0, train=False)


Processed article 1/25: Summary generated.
Processed article 2/25: Summary generated.
Processed article 3/25: Summary generated.
Processed article 4/25: Summary generated.
Processed article 5/25: Summary generated.
Processed article 6/25: Summary generated.
Processed article 7/25: Summary generated.
Processed article 8/25: Summary generated.
Processed article 9/25: Summary generated.
Processed article 10/25: Summary generated.
Processed article 11/25: Summary generated.
Processed article 12/25: Summary generated.
Processed article 13/25: Summary generated.
Processed article 14/25: Summary generated.
Processed article 15/25: Summary generated.
Processed article 16/25: Summary generated.
Processed article 17/25: Summary generated.
Processed article 18/25: Summary generated.
Processed article 19/25: Summary generated.
Processed article 20/25: Summary generated.
Processed article 21/25: Summary generated.
Processed article 22/25: Summary generated.
Processed article 23/25: Summary generate

In [ ]:
summaries_obs_test.head()

,id,article,original_abstract,generated_abstract
0,28278130,Cardiometabolic Risk Factors Among 1.3 Million...,None,Although obesity is a major risk factor for ty...
1,34555924,Prostate Cancer Screening and Incidence among ...,None,Antiretroviral therapy (ART) has transformed t...
2,35157313,The associated burden of mental health conditi...,None,The associated burden of mental health conditi...
3,36906849,Implementing digital systems to facilitate gen...,None,Genetic testing for hereditary breast and ovar...
4,37226713,Patient-reported treatment response in chronic...,None,Chronic graft-versus-host disease (GvHD) is th...


In [ ]:
summaries_rct_test = generate_summaries_dataframe(data_rct_test, tokenizer, model, device, num_articles=25, min_length=50, max_length=120, length_penalty=0.5, verbose=0, train=False)


Processed article 1/25: Summary generated.
Processed article 2/25: Summary generated.
Processed article 3/25: Summary generated.
Processed article 4/25: Summary generated.
Processed article 5/25: Summary generated.
Processed article 6/25: Summary generated.
Processed article 7/25: Summary generated.
Processed article 8/25: Summary generated.
Processed article 9/25: Summary generated.
Processed article 10/25: Summary generated.
Processed article 11/25: Summary generated.
Processed article 12/25: Summary generated.
Processed article 13/25: Summary generated.
Processed article 14/25: Summary generated.
Processed article 15/25: Summary generated.
Processed article 16/25: Summary generated.
Processed article 17/25: Summary generated.
Processed article 18/25: Summary generated.
Processed article 19/25: Summary generated.
Processed article 20/25: Summary generated.
Processed article 21/25: Summary generated.
Processed article 22/25: Summary generated.
Processed article 23/25: Summary generate

In [ ]:
# Merge both summaries_train and summaries_test DataFrames
summaries_all = pd.concat([summaries_obs_test, summaries_rct_test], ignore_index=True)

summaries_all.drop(columns=['original_abstract', 'article'], inplace=True)
summaries_all.rename(columns={'generated_abstract': 'abstract'}, inplace=True)

summaries_all.head()



,id,abstract
0,28278130,Although obesity is a major risk factor for ty...
1,34555924,Antiretroviral therapy (ART) has transformed t...
2,35157313,The associated burden of mental health conditi...
3,36906849,Genetic testing for hereditary breast and ovar...
4,37226713,Chronic graft-versus-host disease (GvHD) is th...


In [ ]:
summaries_all["abstract"] = summaries_all["abstract"].apply(lambda x: f'"{x}"')


In [ ]:
summaries_all.head()

,id,abstract
0,28278130,"""Although obesity is a major risk factor for t..."
1,34555924,"""Antiretroviral therapy (ART) has transformed ..."
2,35157313,"""The associated burden of mental health condit..."
3,36906849,"""Genetic testing for hereditary breast and ova..."
4,37226713,"""Chronic graft-versus-host disease (GvHD) is t..."


In [ ]:
summaries_all.to_csv("summaries_all.csv", index=False)


## **Conclusion Générale**

Dans ce projet, nous avons développé un système de résumé automatique basé sur un modèle encodeur-décodeur, plus précisément BART-large-CNN, pour générer des résumés d'articles scientifiques. L'objectif était de produire des résumés cohérents et informatifs, tout en optimisant les scores de qualité mesurés par la métrique ROUGE-2. 

### **Analyse et interprétation des résultats :**
- **Scores ROUGE-2** :
  - Les résultats sur les données d'entraînement ont donné un score ROUGE-2 moyen d’environ **0.20**, ce qui indique un bon équilibre entre précision et rappel.
  - Sur le jeu de test, soumis lors de la compétition Kaggle, le score ROUGE-2 obtenu était de **0.164**, un résultat solide compte tenu de la complexité de la tâche et des données.
  - Ces performances montrent que le modèle a su capturer l’essentiel des informations tout en restant fidèle au format et au contenu attendu pour les résumés.

- **Impact des hyperparamètres** :
  - Nous avons ajusté les longueurs minimale et maximale des résumés générés (100 à 300 tokens) pour optimiser les résultats. Ce choix s’est avéré crucial, car ces paramètres influencent directement la précision et le rappel :
    - Une longueur minimale trop faible réduit le rappel, tandis qu’une longueur maximale trop élevée diminue la précision.
  - Ce réglage minutieux a permis de maintenir un bon équilibre entre la couverture des informations et la pertinence des résumés.

### **Forces et limites du modèle :**
- **Points forts** :
  - L’utilisation de BART-large-CNN, fine-tuné pour des tâches de summarization, s’est montrée efficace pour produire des résumés pertinents à partir d’articles scientifiques longs et complexes.
  - Les résultats obtenus démontrent que notre pipeline, incluant le découpage des articles en segments avec chevauchement et les ajustements des hyperparamètres, a permis de maximiser les scores.

- **Limites et axes d’amélioration** :
  - **Exploration d’autres modèles** : Des modèles comme T5 ou Pegasus, spécifiquement conçus pour le résumé, pourraient potentiellement offrir de meilleures performances sur ce type de données.
  - **Ajustement des hyperparamètres** : Une exploration plus approfondie des paramètres, comme la pénalité de longueur ou le nombre de faisceaux (`num_beams`) pour la génération, pourrait encore améliorer les résultats.
  - **Préprocessing des données** : Des techniques supplémentaires, comme le nettoyage des données ou la réduction des informations redondantes dans les articles, pourraient aider le modèle à mieux capter l’essentiel.
  - **Fine-tuning spécifique** : Adapter le modèle sur un corpus de données médicales proche des articles OBS et RCT aurait permis une meilleure généralisation.

### **Conclusion finale :**
Les résultats obtenus, avec un score ROUGE-2 de **0.164** sur le jeu de test Kaggle, confirment l’efficacité de notre approche pour cette tâche complexe. Bien que des améliorations soient possibles, ces performances montrent que le modèle encodeur-décodeur choisi, associé à une bonne gestion des hyperparamètres et des segments, a permis d’atteindre un bon compromis entre précision et rappel. 

À l’avenir, l’exploration de modèles alternatifs, un fine-tuning plus ciblé et des optimisations supplémentaires pourraient encore améliorer la qualité des résumés générés, ouvrant ainsi la voie à des solutions plus robustes pour des applications similaires.
